## winpythonの直下にyolov8_ocdフォルダを作成しておく


## githubからクローンするかZIPをダウンロード
https://github.com/atstuyuki/ultralytics
のZIPフォルダをダウンロードして解凍し、中身をすべてyolov8フォルダに移す

## 初期設定

In [2]:
#VS codeの中で現在のディレクトリを確認
#yolov8フォルダに移動
import os
path = os.getcwd()
print(path)

c:\Users\User\Desktop\WPy64-31090\yolov8


## 必要なライブラリをインストール初回のみ実行

In [2]:
#必要なライブラリをインストールする
!pip install -r requirements.txt

^C


In [3]:
#pysimpleGUIをインストール
!pip install pysimplegui

## yolov8の動作確認


In [3]:
#試験検出　学習済みモデルはyolov8xとする　最初はダウンロードに時間がかかる
from ultralytics import YOLO
model = YOLO("yolov8x.pt") 
results = model('zidane.jpg',save=False,save_txt=False,max_det=5)

c:\Users\User\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ultralytics YOLOv8.0.47  Python-3.10.9 torch-1.13.1+cpu CPU
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

image 1/1 C:\Users\User\Desktop\WPy64-31090\yolov8\zidane.jpg: 384x640 2 persons, 1 tie, 1916.1ms
Speed: 5.0ms preprocess, 1916.1ms inference, 25.0ms postprocess per image at shape (1, 3, 640, 640)


## pysimpleGUIによるwebカメラからの検出
# 最初にカメラ番号(0,1,2 のどれか)を半角数字で入力
# 続いて学習済みモデルを選択　(xxx.ptとなっているファイル)
# startで検出開始　thresholdで検出閾値変更　

In [6]:
# ライブラリのimport
from ultralytics import YOLO
import torch
import os
import cv2
from ultralytics.yolo.data.augment import LetterBox
from ultralytics.yolo.utils.plotting import Annotator, colors
from ultralytics.yolo.utils import ops
#from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import PySimpleGUI as sg
import time
import datetime as dt

#すべての警告の非表示
import warnings
warnings.simplefilter('ignore')
from tkinter import *
#tkinterでモニタのサイズを取得
root = Tk()
monitor_height = root.winfo_screenheight()
monitor_width = root.winfo_screenwidth()
root.withdraw()

#閾値の初期設定
threshould = 0.4 

#カメラ番号入力のポップアップ 外部入力なら1か2
cameranum = sg.popup_get_text("input camera number(0,1,2)", title="camera number", default_text="1")
cameranum =int(cameranum)

# 動作させる学習済みモデルを選択
model = sg. popup_get_file("学習済みモデル(xxx.pt)を選択")
model = os.path.basename(model)
sg.popup(model)

#GUIの初期設定
sg.theme('DarkBlue15')
layout = [
   [sg.Image(key='img1',), sg.Image(key='img2',)],
   #[[sg.Text('select model'),sg.Listbox(model_list, size=(10, 3),key=('model_list'))]],

   [[sg.Text('confidence thredshold'),sg.Slider(range=(0.1,1.0),default_value=0.4, resolution=0.1 ,orientation='h',
   size=(20,2),enable_events=True, key='slider',)]],
    [[sg.Button('Start', size=(10, 1)), sg.Button('Stop', size=(10,1))]],

    [sg.Button('Exit', size=(10, 1))],]

#記録イベントのためにstartedという関数を定義               
started= False

#webカメラで検出
#pysimpleGUIのwindow定義 windowサイズはモニタサイズから自動取得

window = sg.Window("webカメラ画面", location=(100,100),layout=layout, size=(monitor_width,monitor_height),resizable=True,finalize=True)
event, values = window.read(timeout=20)

cap = cv2.VideoCapture(cameranum) #外部入力カメラを使用する場合0を1に変更
# ビデオ記録用の変数定義
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('result_{}.mp4'.format(dt.datetime.now()), fourcc, fps, (width, height))
model = YOLO(model)
while True:
    ret, imgs = cap.read()
    if not ret:
        break
    #imageを小さくする
    imgs = cv2.resize(imgs, (int(monitor_width/2),int(monitor_height/2)))
    h,w,ch=imgs.shape
# キャプチャーした画像をpngに変換
    image1 = cv2.imencode('.png', imgs)[1].tobytes()                   
# Imageの内容を更新
    window['img1'].update(data=image1)#web cameraの画像を直接表示
    event, values = window.read(timeout=0)

    if values['slider']:
      threshould=values['slider']
    

    if event == 'Start':
        started = True
    if started == True:
        #画面を10*10分割して1/10のところから9/10のところまで切り抜き
        im_cropped=imgs[round(h/10):round(h/10)*9, round(w/10):round(w/10)*9,:]
        #切り抜いた画像をモニターのサイズに合わせて拡大
        im_cropped=cv2.resize(im_cropped,(int(monitor_width/2),int(monitor_height/2)))
        #切り抜いた画像に対して推論
        preds = model(im_cropped, conf=threshould)
        #この操作で推論結果をnaddaryに変換して画像表示
        preds_plotted = preds[0].plot()
    # キャプチャーした画像をpngに変換
        image2 = cv2.imencode('.png', preds_plotted)[1].tobytes()
    # Imageの内容を更新
        window['img2'].update(data=image2)       

    if event =='Stop':
        started = False
        out.release()

    if event == 'Exit' or event == sg.WIN_CLOSED:
        break

cap.release()    
window.close()

Ultralytics YOLOv8.0.47  Python-3.10.9 torch-1.13.1+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 448x640 1 person, 202.4ms
Speed: 2.0ms preprocess, 202.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 person, 191.1ms
Speed: 1.0ms preprocess, 191.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 person, 1 book, 189.1ms
Speed: 1.0ms preprocess, 189.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 person, 189.1ms
Speed: 1.0ms preprocess, 189.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 person, 193.1ms
Speed: 2.0ms preprocess, 193.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 person, 188.4ms
Speed: 1.0ms preprocess, 188.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 1 person, 174.5ms
Speed: 1.0ms preprocess, 174.5ms inference, 2.0ms pos